In [ ]:
# this file builds a graph of psych keywords from filtered & combined data (.pkl)
# require: ../data/combined_data.pkl
#          ../dictionary/freq_dict_162.txt

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter
import ast
import math

In [2]:
# # read data from csv
# filename = "../data/combined_data.csv" # "../data/toy_data.csv"
# pubmed = pd.read_csv(filename)
# # convert string representation back to list
# string_represent_variables = ['tokenized_abstract', 'tokenized_title', 
#                               'normalized_abstract', 'normalized_title', 
#                               'psyc_check']
# for variable in string_represent_variables:
#     pubmed[variable] = pubmed[variable].apply(ast.literal_eval)
# pubmed # dim 111571 22

In [3]:
# read data from pickle (15 seconds)
# pubmed.to_pickle("../data/combined_data.pkl")
pubmed = pd.read_pickle("/Users/lyumu/Google Drive/pubmed_preprocessing/combined_data.pkl")
pubmed # dim 111571 22

,abstract,affiliation,author,country,delete,issn_linking,journal,keywords,medline_ta,mesh_terms,...,pmc,pmid,pubdate,title,tokenized_abstract,tokenized_title,normalized_abstract,normalized_title,psyc_check,psyc_filter
0,Both gamma-glutamyltransferase (gammaGT) and p...,NaN,M Cucuianu; D Zdrenghea; M Pop; A Opincaru,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,NaN,Clin Chim Acta,D000328:Adult; D000368:Aged; D000437:Alcoholis...,...,NaN,9218,1976,Increased serum gamma-glutamyltransferase in h...,"[Both, gamma-glutamyltransferase, (, gammaGT, ...","[Increased, serum, gamma-glutamyltransferase, ...","[gammagt, pseudocholinesteras, pce, found, inc...","[increas, serum, hypertriglyceridemia, compari...",[behavior],True
1,Hearing impairment and related cochlear histop...,NaN,WC Stebbins; JE Hawkins; LG Johnson; DB Moody,United States,False,0196-0709,American journal of otolaryngology,NaN,Am J Otolaryngol,D000818:Animals; D001299:Audiometry; D001309:A...,...,NaN,95382,1979,Hearing thresholds with outer and inner hair c...,"[Hearing, impairment, and, related, cochlear, ...","[Hearing, thresholds, with, outer, and, inner,...","[hear, impair, relat, cochlear, histopatholog,...","[hear, threshold, outer, inner, hair, cell, loss]",[behavior],True
2,RBP behavior was studied in different kidney d...,NaN,L Scarpioni; PP Dall'aglio; PG Poisetti; C Buzio,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,NaN,Clin Chim Acta,D003404:Creatinine; D006801:Humans; D007674:Ki...,...,NaN,944115,1976,Retinol binding protein in serum and in urine ...,"[RBP, behavior, was, studied, in, different, k...","[Retinol, binding, protein, in, serum, and, in...","[rbp, behavior, studi, differ, kidney, diseas,...","[retinol, bind, protein, serum, urin, glomerul...",[behavior],True
3,The determination of Zn in fingernails directl...,NaN,A Sohler; P Wolcott; CC Pfeiffer,Netherlands,False,0009-8981,Clinica chimica acta; international journal of...,NaN,Clin Chim Acta,D005069:Evaluation Studies as Topic; D006197:H...,...,NaN,947632,1976,Determination of zinc in fingernails by non-fl...,"[The, determination, of, Zn, in, fingernails, ...","[Determination, of, zinc, in, fingernails, by,...","[determin, zn, fingernail, direct, use, graphi...","[determin, zinc, fingernail, atom, absorpt, sp...",[behavior],True
4,PURPOSE\r\nThis study was developed to test th...,"Department of Psychiatry, Cornell University M...",B Landis; E Landis,United States,False,0196-0709,American journal of otolaryngology,NaN,Am J Otolaryngol,D000328:Adult; D000368:Aged; D001299:Audiometr...,...,NaN,1443390,1992,Is biofeedback effective for chronic tinnitus?...,"[PURPOSE, This, study, was, developed, to, tes...","[Is, biofeedback, effective, for, chronic, tin...","[purpos, studi, develop, test, hypothesi, inte...","[biofeedback, effect, chronic, tinnitus, inten...","[psycholog, psycholog]",True
5,INTRODUCTION\r\nRecurrent otitis media is comm...,"Department of Otorhinolaryngology, University ...",M Zargi; IH Boltezar,United States,False,0196-0709,American journal of otolaryngology,NaN,Am J Otolaryngol,D001307:Auditory Perception; D002648:Child; D0...,...,NaN,1443393,1992,Effects of recurrent otitis media in infancy o...,"[INTRODUCTION, Recurrent, otitis, media, is, c...","[Effects, of, recurrent, otitis, media, in, in...","[introduct, recurr, otiti, media, common, enco...","[effect, recurr, otiti, media, infanc, auditor...",[behavior],True
6,This study examines the very early stages of o...,"Division of Head and Neck Surgery, Harbor-UCLA...",RF Canalis; FD Burstein; P Dickman; G Berke,United States,False,0196-0709,American journal of otolaryngology,NaN,Am J Otolaryngol,D000818:Animals; D001842:Bone and Bones; D0021...,...,NaN,1858978,1991,Early structural changes in vascularized perio...,"[This, study, examines, the, very, early, stag...","[Early, structural, changes, in, vascularized,...","[studi, examin, veri, earli, stage, osteogenes...","[

In [7]:
pubmed_16to18 = pubmed.query('pubdate >= "2016" and pubdate <= "2018"')

In [4]:
# define dictionary
with open("../dictionary/psyc_unique_dict_847_2k-1k.txt") as f:
    psyc_dict = [x.strip('\n') for x in f.readlines()]
psyc_dict = sorted(psyc_dict) # sort alphabetically
psyc_dict # len 172

['aberr',
 'abstin',
 'abstract',
 'abus',
 'academ',
 'acceler',
 'accompani',
 'accumben',
 'acoust',
 'acquisit',
 'actual',
 'addict',
 'adequ',
 'adhd',
 'adher',
 'adhes',
 'adolesc',
 'adopt',
 'adsorpt',
 'adulthood',
 'affin',
 'african',
 'aggreg',
 'aggress',
 'agonist',
 'aid',
 'air',
 'alcohol',
 'align',
 'allevi',
 'alloc',
 'alreadi',
 'alway',
 'alzheim',
 'amelior',
 'american',
 'amplitud',
 'amygdala',
 'analog',
 'anatom',
 'angl',
 'anion',
 'answer',
 'antagonist',
 'anterior',
 'anticip',
 'antidepress',
 'antioxid',
 'anxieti',
 'appar',
 'aqueous',
 'architectur',
 'argu',
 'aris',
 'arm',
 'arous',
 'array',
 'artifici',
 'assembl',
 'assist',
 'assum',
 'assumpt',
 'atom',
 'attempt',
 'attend',
 'attitud',
 'attract',
 'auditori',
 'augment',
 'autism',
 'automat',
 'autonom',
 'avers',
 'awar',
 'axi',
 'axon',
 'balanc',
 'band',
 'basal',
 'basic',
 'batteri',
 'behav',
 'belief',
 'believ',
 'belong',
 'beyond',
 'bias',
 'bilater',
 'biochem',
 'biome

In [5]:
def countOccurrence(text, targetWords):
    frequency = Counter(text)
    targetFrequency = {word: frequency[word] for word in targetWords}
    return pd.Series(targetFrequency) # call Series s.t. the apply function returns a dataframe

In [8]:
# count occurrence (40 seconds / len=10) (1 min / len=172)
psyc_occurrence = pubmed_16to18['normalized_abstract'].apply(countOccurrence, targetWords=psyc_dict)
psyc_occurrence.insert(loc=0, column='pmid', value=pubmed.pmid)
psyc_occurrence

,pmid,aberr,abstin,abstract,abus,academ,acceler,accompani,accumben,acoust,...,widespread,wild,wiley,window,withdraw,word,worker,wors,younger,youth
312,23283758,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
321,24149941,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
322,24214969,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
323,24376045,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
325,24505061,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326,24799318,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
329,24935694,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
330,24970718,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331,25018083,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
334,25112818,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# build adjacency matrix (10 seconds / len=172)
len_dict = len(psyc_dict)
adj_matrix = np.zeros((len_dict, len_dict), dtype=np.int64)
for row in psyc_occurrence.loc[:, psyc_dict].itertuples():
    positive_freq_idx = [idx for (idx, value) in enumerate(row[1:]) if value > 0]
    for i in positive_freq_idx:
        for j in positive_freq_idx:
            if i >= j:
                continue
            adj_matrix[i, j] += 1
            # print(i,j)
adj_matrix = adj_matrix + adj_matrix.T
adj_matrix

array([[ 0, 12,  4, ...,  8, 11, 12],
       [12,  0,  4, ..., 10,  9,  7],
       [ 4,  4,  0, ..., 12, 44, 24],
       ...,
       [ 8, 10, 12, ...,  0, 54, 43],
       [11,  9, 44, ..., 54,  0, 90],
       [12,  7, 24, ..., 43, 90,  0]], dtype=int64)

In [10]:
# log-transform the edge weights
log_adj_matrix = np.log(adj_matrix + 1)
log_adj_matrix

array([[0.        , 2.56494936, 1.60943791, ..., 2.19722458, 2.48490665,
        2.56494936],
       [2.56494936, 0.        , 1.60943791, ..., 2.39789527, 2.30258509,
        2.07944154],
       [1.60943791, 1.60943791, 0.        , ..., 2.56494936, 3.80666249,
        3.21887582],
       ...,
       [2.19722458, 2.39789527, 2.56494936, ..., 0.        , 4.00733319,
        3.78418963],
       [2.48490665, 2.30258509, 3.80666249, ..., 4.00733319, 0.        ,
        4.51085951],
       [2.56494936, 2.07944154, 3.21887582, ..., 3.78418963, 4.51085951,
        0.        ]])

In [11]:
H, bins = np.histogram(log_adj_matrix)
plt.bar(bins[:-1], H, width=1)

<BarContainer object of 10 artists>

In [12]:
# build graph from adjacency matrix
# g = nx.convert_matrix.from_numpy_matrix(adj_matrix)
# g = nx.convert_matrix.from_numpy_matrix(adj_matrix / adj_matrix.max())
# g = nx.convert_matrix.from_numpy_matrix(log_adj_matrix)
g = nx.convert_matrix.from_numpy_matrix(log_adj_matrix / log_adj_matrix.max())
g = nx.relabel_nodes(g, {i : word for i, word in enumerate(psyc_dict)})

In [13]:
# len(g.nodes), len(g.edges)
nx.info(g)

'Name: \nType: Graph\nNumber of nodes: 847\nNumber of edges: 327255\nAverage degree: 772.7391'

In [14]:
# output to Gephi
nx.write_gexf(g, "../gephi/pubmed-847-log-normalized-16to18.gexf")

In [15]:
# draw graph (too slow to run on len=172)
# pos = nx.spring_layout(g, iterations=50)
# labels = nx.get_edge_attributes(g, 'weight')
# nx.draw_networkx(g, pos)
# nx.draw_networkx_edge_labels(g, pos, edge_labels=labels)